In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd

import xgboost as xgb
print("XGBoost version:", xgb.__version__)

import warnings
warnings.filterwarnings("ignore")
import datatable as dt
from sklearn.impute import SimpleImputer

In [ ]:
print('Loading...')
train = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv').to_pandas()
print("Done!")

In [ ]:
train_dataset_weight = train[train['weight'] != 0]
train_dataset_weight['action'] = ( train_dataset_weight['resp'].values > 0).astype(int)

# Imputation
my_imputer = SimpleImputer(strategy='mean')

imputed_train_dataset_weight = pd.DataFrame(my_imputer.fit_transform(train_dataset_weight))

imputed_train_dataset_weight.columns = train_dataset_weight.columns
del train_dataset_weight
del my_imputer
del train

In [ ]:
y = imputed_train_dataset_weight.action

In [ ]:
selected_features = ['feature_43', 'feature_42', 'feature_45', 'feature_41', 'feature_44', 'feature_63', 'feature_61', 'feature_69', 'feature_6', 'feature_64', 'feature_5', 'feature_62', 'feature_7', 'feature_11', 'feature_20', 'feature_39', 'feature_60', 'feature_83', 'feature_3', 'feature_37', 'feature_1', 'feature_40', 'feature_4', 'feature_38', 'feature_27', 'feature_77', 'feature_119', 'feature_28', 'feature_120', 'feature_68', 'feature_95', 'feature_90', 'feature_66', 'feature_55', 'feature_121', 'feature_89', 'feature_84', 'feature_107', 'feature_114', 'feature_113', 'feature_71', 'feature_8', 'feature_124', 'feature_49', 'feature_101', 'feature_125', 'feature_102', 'feature_78', 'feature_57', 'feature_67', 'feature_65', 'feature_108', 'feature_70', 'feature_31', 'feature_48', 'feature_126', 'feature_18', 'feature_26', 'feature_96', 'feature_86', 'feature_116', 'feature_127', 'feature_22', 'feature_92', 'feature_51', 'feature_58', 'feature_12', 'feature_33', 'feature_53', 'feature_17', 'feature_104', 'feature_24', 'feature_110', 'feature_72', 'feature_36', 'feature_21', 'feature_35', 'feature_32', 'feature_25', 'feature_59', 'feature_34', 'feature_2', 'feature_93', 'feature_10', 'feature_117', 'feature_87', 'feature_47', 'feature_128', 'feature_98', 'feature_80', 'feature_50', 'feature_54', 'feature_79', 'feature_129', 'feature_9', 'feature_23', 'feature_19', 'feature_111', 'feature_115', 'feature_56', 'feature_30', 'feature_73', 'feature_122', 'feature_105', 'weight', 'feature_74', 'feature_14', 'feature_123', 'feature_29', 'feature_109', 'feature_82', 'feature_112', 'feature_76', 'feature_16', 'feature_99', 'feature_88', 'feature_85', 'feature_106', 'feature_118', 'feature_91', 'feature_46', 'feature_81', 'feature_75', 'feature_94', 'feature_52', 'feature_13', 'feature_103', 'feature_15', 'feature_100', 'feature_97', 'feature_0']

In [ ]:
X_model = imputed_train_dataset_weight[selected_features]
del imputed_train_dataset_weight

In [ ]:
print('Creating classifier...', end='')
clf = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=11,
    eta=0.06, # learning_rate
    missing=None,
    random_state=0,
    tree_method='gpu_hist',
    subsample=0.85,
    colsample_bytree=0.6,
    #sampling_method='gradient_based',
    #eval_metric='logloss',
    verbosity=2   # info
)
print('Finished.')

In [ ]:
print('Training classifier...', end='')
%time clf.fit(X_model, y)
print('Finished.')
del X_model
del y

In [ ]:
# Submitting to Jane Street

import janestreet
env = janestreet.make_env()
env_iter = env.iter_test()

print('Creating submissions file...', end='')
for (test_df, prediction_df) in env.iter_test():
    X_test = test_df.loc[:, selected_features]
    y_preds = clf.predict(X_test)
    prediction_df.action = y_preds.astype(int)
    env.predict(prediction_df)